<a href="https://colab.research.google.com/github/strumer69/SGD_classifier_OpenCV_/blob/main/chess_peice_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# chess peice detection (generate fen)
* this code generates fen code according to location of chess peices and by the use of SGDClassifier machine learning model.
* the method is the same as the famous MNIST problem
* fen means the codes which represents the exact peices of chess in each team. for axample R represent white rook, q means the black queen and etc.
* more info about fen: [fen_wiki](https://en.wikipedia.org/wiki/Forsyth%E2%80%93Edwards_Notation)

* the training data contains 18k chess board ( [G_drive](https://drive.google.com/file/d/1LQw3tHxbDzXJuZjcwJrwcJHzsYsp9ZfH/view?usp=share_link))
* the data for trainig the mode(60k-peice)( [G_drive](https://drive.google.com/file/d/1Elel68b2BhUqOxyytonA8iJeA-VL84sJ/view?usp=sharing) )
* the test data (the answers should be checked visually)( [G_Drive](https://drive.google.com/file/d/1Af65xO6ygKiX4TYeUpXbacGjDEvI6OZJ/view?usp=share_link))

# import libraries

In [ ]:
import cv2
import numpy as np
import pandas as pd
import glob
import os
import re
from tqdm import tqdm
import matplotlib as mpl
import matplotlib.pyplot as plt

# Introduction of the folder which contains images

In [ ]:
CHESS_BOARD_DIR = glob.glob('train/*.*')
CHESS_BOARD = []

# define a function to normalize fen code


In [ ]:
def normalizeBoardName(boardName):
    mine_code=[]
    longName=boardName.split('-')
    for word in longName:
        for letter in word:
            if letter.isnumeric():
                for i in range(int(letter)):
                    mine_code.append(str(0))
            else:
                mine_code.append(letter)
                
    return mine_code

# creation of 60000 images from chess board
each board contains 64 square that at most 32 peices exist in game

In [ ]:

df = pd.DataFrame()
points=[5,19,  20,34,  35,49,  50,64,  64,78,  79,93,  94,108,  109,123]
my_counter=0
for idx, path in enumerate(CHESS_BOARD_DIR):
    my_counter_local=0
    if idx==1000:
        break
    
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    CHESS_BOARD = [path[6:-4], image]
    PEACE_CODES=normalizeBoardName(CHESS_BOARD[0])
    #--------------------------------------------
    
    for j in range(0,len(points) , 2):# columns
        
        for i in range (0,len(points) , 2):# rows
            for board in CHESS_BOARD:
                img_small = CHESS_BOARD[1]
                img_small=img_small[points[j]:points[j+1],points[i]:points[i+1]] #first col , then row
                
#         cv2.imshow('s', img_small)
#         cv2.waitKey(0)
#         cv2.destroyAllWindows()
#         print(type(img_small))


            img_small = img_small.reshape(14*14)
            img_small=np.append(img_small,PEACE_CODES[my_counter_local])
            df[my_counter]=img_small
            my_counter+=1
            my_counter_local+=1



# save the 60k images in a excel file

In [ ]:
# df=df.T
# df.head()
df.to_csv('all_images2.csv', sep=',')

# reading the data from directory

In [ ]:
df_1=pd.read_csv('all_images2.csv')


# EDA (Exploratory Data Analysis) and Data Cleaning

In [ ]:
df_1.shape

In [ ]:
df_1.head()

In [ ]:
df_1= df_1.drop('Unnamed: 0',axis=1)


In [ ]:
df_1.head()

In [ ]:
df_1.shape

In [ ]:
X = df_1.drop('196', axis=1)
print(X.shape)

In [ ]:
X.head()

In [ ]:
X.info()

In [ ]:
X=X.astype(np.int64)

In [ ]:
X.info()

In [ ]:
type(X)

In [ ]:
X.head()

In [ ]:
type(X.head(1))

In [ ]:
y = df_1['196']
type(y)

In [ ]:
print(y.shape)

In [ ]:
y.unique()

In [ ]:
x_array=X. to_numpy()

some_digit =x_array[1]

In [ ]:
x_array.shape

In [ ]:
some_digit_image = some_digit.reshape(14, 14)
print(type(some_digit_image))

In [ ]:
plt.imshow(some_digit_image,cmap="binary")
plt.axis("off")
plt.show()


# convert categorical feature to numerical
we use label encoder to convert the fen values (which are alphabets) to numerical values.

In [ ]:

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
label = le.fit_transform(df_1['196'])
label


In [ ]:
label.shape

In [ ]:
import numpy as np
y=label

y = y.astype(np.uint8)

In [ ]:
X_train, X_test, \
y_train, y_test =\
X[:60000], X[60000:], y[:60000], y[60000:]

In [ ]:
from sklearn.linear_model import SGDClassifier
sgd_clf = SGDClassifier(random_state=42)
sgd_clf.fit(X_train, y_train)

now our model is trained and we can give it a sample and take prediction. for example if we give the mode an image rook it will return r. or if we give it knight it will return N ( as k is reserved for king)

In [ ]:
print(X.iloc[2])

In [ ]:
type(X.loc[2])

In [ ]:
pd.DataFrame(X.loc[0]).T

In [ ]:
pd.DataFrame(X.loc[0]).T.shape

In [ ]:
# sgd_clf.predict(X.head())
sgd_clf.predict(pd.DataFrame(X.loc[1]).T)


In [ ]:
le.inverse_transform(sgd_clf.predict(pd.DataFrame(X.loc[1]).T))

In [ ]:
le.inverse_transform(sgd_clf.predict(X.head()))

In [ ]:
X.shape

# prediction on the main dataset that is related to chess peice detection competition

In [ ]:
import cv2
import numpy as np
import glob
import os
import re
from tqdm import tqdm

In [ ]:
#CHESS_BOARD_DIR = glob.glob('train/*.*')
# CHESS_BOARD_DIR = glob.glob('test/*.*')
CHESS_BOARD_DIR = glob.glob('test/*.*')
# CHESS_BOARD_DIR

In [ ]:
chessBoardImages = dict()
chessBoardImages 

for idx, path in enumerate(CHESS_BOARD_DIR):
#     if idx > 1:
#         break
    
    baseName = os.path.basename(path)
#     print(baseName)
    fileName = re.search('[\w() -]+?(?=\.)', baseName).group(0)#جدا کردن نام از پسوند 
    boardImage = cv2.imread(path)
    chessBoardImages[fileName] = boardImage

In [ ]:
def chessDetection(boardImage):
    image = boardImage
    imageG = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    points = [5, 19,  20, 34,  35, 49,  50, 64,
              64, 78,  79, 93,  94, 108,  109, 123]
    all = []
    cols = []
    
    for j in range(0, len(points), 2):
        for i in range(0, len(points), 2): 

            img_small = imageG[points[j]:points[j+1],points[i]:points[i+1]]  
            img_big = cv2.resize(img_small, (0, 0), fx=10, fy=10)


            img_small_2=img_small.reshape(1,196)

            cols.append(le.inverse_transform(sgd_clf.predict(img_small_2))[0])
        all.append(cols)
        cols=[]
        
    outt=''
    numb=0

    for line in all:

        for item in line:
            if item=='0':
                numb+=1
            else :
                if numb!=0:
                    outt+=str(numb)
                outt+=item
                numb=0
        if numb!=0:
            outt+=str(numb)
        numb=0
        outt+='-'
    # print(outt[0:-1])
    return outt[0:-1]
    numb=0

In [ ]:
# for boardName in chessBoardImages:
#     fen = chessDetection(chessBoardImages[boardName])
#     print('detect=> ',fen)
#     print('real=>   ', boardName)
#     print(fen == boardName)

count = 0
wrongs=0
problem = []

f = open("predictions2.csv", "w")
f.write('image,label')
for boardName in chessBoardImages:
    fen = chessDetection(chessBoardImages[boardName])
    f.write('\n' + boardName + ',' + fen)
    
    count += 1
#     if fen != boardName:
#         wrongs+=1
#         print(wrongs)
#         problem.append((boardName, fen))
#         print('True =',boardName)
#         print('False=',fen)
#         print('-----')
#     else:
#         print('True   =',boardName)
#         print('predict=',fen)
#         print('-----')
    
f.close()
print('done')
# print("count of images: ", count, "\n wrong: ", problem)

done
